In [1]:
!pip install keras

In [33]:
import pandas as pd
import numpy as np
import json
from keras.models import load_model
myMastery = {
  "1": 7154,
  "2": 11627,
  "3": 18755,
  "4": 0,
  "5": 2994,
  "6": 13918,
  "7": 83,
  "8": 3430,
  "9": 7353,
  "10": 5572,
  "11": 133,
  "12": 48454,
  "13": 924,
  "14": 55191,
  "15": 0,
  "16": 8532,
  "17": 7648,
  "18": 12204,
  "19": 6716,
  "20": 7218,
  "21": 15437,
  "22": 10957,
  "23": 5620,
  "24": 26390,
  "25": 3945,
  "26": 1734,
  "27": 1307,
  "28": 9448,
  "29": 7882,
  "30": 22154,
  "31": 5495,
  "32": 9702,
  "33": 2842,
  "34": 1476,
  "35": 1578,
  "36": 9377,
  "37": 6796,
  "38": 2373,
  "39": 165,
  "40": 3590,
  "41": 12284,
  "42": 1554,
  "43": 1385,
  "44": 10893,
  "45": 30008,
  "48": 15394,
  "50": 4276,
  "51": 414,
  "53": 20291,
  "54": 16350,
  "55": 161,
  "56": 3413,
  "57": 10246,
  "58": 13995,
  "59": 3797,
  "60": 164,
  "61": 0,
  "62": 1440,
  "63": 6223,
  "64": 5646,
  "67": 11633,
  "68": 4843,
  "69": 191,
  "72": 3081,
  "74": 1223,
  "75": 10987,
  "76": 393,
  "77": 1277,
  "78": 27807,
  "79": 4717,
  "80": 12383,
  "81": 15481,
  "82": 18925,
  "83": 634,
  "84": 288,
  "85": 25669,
  "86": 14323,
  "89": 5605,
  "90": 2937,
  "91": 1131,
  "92": 256,
  "96": 2153,
  "98": 8616,
  "99": 2343,
  "101": 4250,
  "102": 5968,
  "103": 6494,
  "104": 2517,
  "105": 1585,
  "106": 2472,
  "107": 3479,
  "110": 4193,
  "111": 3886,
  "112": 3471,
  "113": 15296,
  "114": 6895,
  "115": 6626,
  "117": 5687,
  "119": 171,
  "120": 7340,
  "121": 9401,
  "122": 5056,
  "126": 7506,
  "127": 1157,
  "131": 685,
  "133": 3653,
  "134": 1022,
  "136": 2244,
  "141": 3882,
  "142": 728,
  "143": 11928,
  "145": 3179,
  "147": 611,
  "150": 3643,
  "154": 16551,
  "157": 3250,
  "161": 10299,
  "163": 653,
  "164": 13701,
  "201": 6966,
  "202": 1068,
  "203": 3867,
  "222": 4678,
  "223": 28993,
  "235": 1893,
  "236": 5951,
  "238": 2748,
  "240": 4782,
  "245": 281,
  "246": 0,
  "254": 3255,
  "266": 23025,
  "267": 2420,
  "268": 0,
  "350": 1294,
  "360": 0,
  "412": 14854,
  "420": 4047,
  "421": 3028,
  "427": 449,
  "429": 1270,
  "432": 14439,
  "497": 10803,
  "498": 3578,
  "516": 13867,
  "517": 866,
  "518": 679,
  "523": 0,
  "555": 8995,
  "777": 0,
  "875": 6780,
  "876": 137
}

In [23]:
def Convert(champList):
    TEAM = len(champList)/2
    TEAM = (int)(TEAM)

    synerge = pd.read_json('./Data/WSC/Synerge.json')
    counter = pd.read_json('./Data/WSC/Counter.json')
    winRate = pd.read_json('./Data/WSC/WinRate.json')
    InputList=[]

    #Red
    for i in range(TEAM):
        champ = champList[i]
        myS=0
        myC=0
        for j in range(TEAM):
            champ2 = champList[j]
            if champ != champ2:
                myS += synerge[champ][champ2]["WinRate"]
            champ2 = champList[j+TEAM]
            myC += counter[champ][champ2]["WinRate"]
        if TEAM>1:
            myS = myS/(TEAM-1)
        myC = myC/TEAM
        InputList.append(winRate[champ]['WinRate'])
        InputList.append(myS-50)
        InputList.append(myC-50)

    #Blue
    for i in range(TEAM):
        i=i+TEAM
        champ = champList[i]
        myS=0
        myC=0
        for j in range(TEAM):
            champ2 = champList[j+TEAM]
            if champ != champ2:
                myS += synerge[champ][champ2]["WinRate"]
            champ2 = champList[j]
            myC += counter[champ][champ2]["WinRate"]
        if TEAM>1:
            myS = myS/(TEAM-1)
        myC = myC/TEAM
        InputList.append(winRate[champ]['WinRate'])
        InputList.append(myS-50)
        InputList.append(myC-50)
    ret = np.array(InputList).reshape((1,TEAM*6))
    return ret


In [30]:
def mastery_filter(masteryDict):
    ID_to_Name = {}
    file = pd.read_csv('./Data/championName_ID.csv')
    for i in range(len(file)):
        ID = file.loc[i]['id']
        name = file.loc[i]['name']
        ID_to_Name[ID]=name
        
    ret = []
    
    for k,v in masteryDict.items():
        if v>12600:
            k = (int)(k)
            name= ID_to_Name[k]
            ret.append(name)
    return ret

In [25]:
def Model_result(Red,Blue,Ban,Pos,MasteryDict):
    """
    Red is a pick list of Red Team (List of ChampionName)
    Blue is a pick list of Blue Team (List of ChampionName)
    Ban : List of ChampionName
    Pos : TOP | MIDDLE | JUNGLE | BOTTOM | UTILITY : str
    
    ret[0]: list, top three champion chosen from all champion pool
    ret[1]: list, top three champion chosen from mastery champion pool
    """
    
    length = len(Red)+len(Blue)+1
    length = (int)(length/2)
    line = json.load(open('./Data/LineInfo/LineInfo.json'))
    winRate = pd.read_json('./Data/WSC/WinRate.json')
    
    
    # if I am the first
    if length ==0:
        rateL=[]
        rateD={}
        for champ in line[Pos]:
            rateL.append(winRate[champ]['WinRate'])
            if winRate[champ]['WinRate'] in rateD.keys():
                rateD[winRate[champ]['WinRate']].append(champ)
            else:
                rateD[winRate[champ]['WinRate']]=[champ]
        rateL.sort(reverse=True)
        
        cnt=0
        ret=[]
        for i in rateL:
            for champ in rateD[i]:
                if winRate[champ]['Win']+winRate[champ]['Lose']>200: # pick rate > 2%
                    ret.append(champ)
                    cnt+=1
                if(cnt>=3):
                    return ret
            
    #Else
    model = load_model('./AImodel/model_'+(str)(length)+'.h5')
    mymax=0
    winC =""
    rankD={}
    rankL=[]
    for champ in line[Pos]:
        
        if champ in Red+Blue+Ban:
            continue
        Red.append(champ)
        Team=Red+Blue
        I = Convert(Team)
        res=model.predict(I)
        if res[0][0] in rankD.keys():
            rankD[res[0][0]].append(champ)
        else:
            rankD[res[0][0]]=[champ]
        rankL.append(res[0][0])
        del Red[-1]
    rankL.sort(reverse=True)
    
    # Output Step
    cnt_0=0 #for all
    cnt_1=0 # mastery
    ret=[[],[]]
    MasteryList = mastery_filter(MasteryDict)
    # 승률 지표  높은 순으로
    for i in rankL:
        # 챔피언을 찾아서
        for champ in rankD[i]:
            #모든 챔피언 풀
            if cnt_0<3:
                if winRate[champ]['Win']+winRate[champ]['Lose']>100: # pick rate > 1%
                    ret[0].append(champ)
                    cnt_0+=1
            
            #숙련도 챔피언 풀
            if cnt_1 <3:
                if champ in MasteryList:
                    if winRate[champ]['Win']+winRate[champ]['Lose']>100: # pick rate > 1%
                        ret[1].append(champ)
                        cnt_1+=1
            if cnt_0>=3 and cnt_1>=3 :
                return ret

In [34]:
res=Model_result(["Amumu", "Annie", "Shen"],["Ornn","Leblanc"],[],"JUNGLE",myMastery)
res

[['Fiddlesticks', 'Elise', 'Sejuani'], ['Sejuani', 'Karthus', 'Jax']]

{'1': 7154,
 '2': 11627,
 '3': 18755,
 '4': 0,
 '5': 2994,
 '6': 13918,
 '7': 83,
 '8': 3430,
 '9': 7353,
 '10': 5572,
 '11': 133,
 '12': 48454,
 '13': 924,
 '14': 55191,
 '15': 0,
 '16': 8532,
 '17': 7648,
 '18': 12204,
 '19': 6716,
 '20': 7218,
 '21': 15437,
 '22': 10957,
 '23': 5620,
 '24': 26390,
 '25': 3945,
 '26': 1734,
 '27': 1307,
 '28': 9448,
 '29': 7882,
 '30': 22154,
 '31': 5495,
 '32': 9702,
 '33': 2842,
 '34': 1476,
 '35': 1578,
 '36': 9377,
 '37': 6796,
 '38': 2373,
 '39': 165,
 '40': 3590,
 '41': 12284,
 '42': 1554,
 '43': 1385,
 '44': 10893,
 '45': 30008,
 '48': 15394,
 '50': 4276,
 '51': 414,
 '53': 20291,
 '54': 16350,
 '55': 161,
 '56': 3413,
 '57': 10246,
 '58': 13995,
 '59': 3797,
 '60': 164,
 '61': 0,
 '62': 1440,
 '63': 6223,
 '64': 5646,
 '67': 11633,
 '68': 4843,
 '69': 191,
 '72': 3081,
 '74': 1223,
 '75': 10987,
 '76': 393,
 '77': 1277,
 '78': 27807,
 '79': 4717,
 '80': 12383,
 '81': 15481,
 '82': 18925,
 '83': 634,
 '84': 288,
 '85': 25669,
 '86': 14323,
 '8

{'Annie': 7154,
 'Olaf': 11627,
 'Galio': 18755,
 'TwistedFate': 0,
 'XinZhao': 2994,
 'Urgot': 13918,
 'Leblanc': 83,
 'Vladimir': 3430,
 'Fiddlesticks': 7353,
 'Kayle': 5572,
 'MasterYi': 133,
 'Alistar': 48454,
 'Ryze': 924,
 'Sion': 55191,
 'Sivir': 0,
 'Soraka': 8532,
 'Teemo': 7648,
 'Tristana': 12204,
 'Warwick': 6716,
 'Nunu': 7218,
 'MissFortune': 15437,
 'Ashe': 10957,
 'Tryndamere': 5620,
 'Jax': 26390,
 'Morgana': 3945,
 'Zilean': 1734,
 'Singed': 1307,
 'Evelynn': 9448,
 'Twitch': 7882,
 'Karthus': 22154,
 'Chogath': 5495,
 'Amumu': 9702,
 'Rammus': 2842,
 'Anivia': 1476,
 'Shaco': 1578,
 'DrMundo': 9377,
 'Sona': 6796,
 'Kassadin': 2373,
 'Irelia': 165,
 'Janna': 3590,
 'Gangplank': 12284,
 'Corki': 1554,
 'Karma': 1385,
 'Taric': 10893,
 'Veigar': 30008,
 'Trundle': 15394,
 'Swain': 4276,
 'Caitlyn': 414,
 'Blitzcrank': 20291,
 'Malphite': 16350,
 'Katarina': 161,
 'Nocturne': 3413,
 'Maokai': 10246,
 'Renekton': 13995,
 'JarvanIV': 3797,
 'Elise': 164,
 'Orianna': 0,
 '